# HR Analytics - Classification Algorithms Pipeline

In this notebook, I examined HR Analytics: Job Change of Data Scientist dataset. Firstly, I visualized the data, then filled missing values and I handled skewness problem. I applied One-Hot Encoding to some features. After train-test split, I applied 9 classification algorithms to this dataset. You will find out train-validation-test scores, confusion matrixes, evaluation metrics and classification reports belongs to every algorithm. Also, I applied Cross Validation to all algorithms. I visualized the results.

## If you have questions please ask them on the comment section.

## I will be glad if you can give feedback.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the Necessary Libraries

In [ ]:
import numpy as np 
import pandas as pd
import pandas
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
import seaborn as sns; sns.set()

from sklearn import tree
import graphviz 
import os
import preprocessing 

import numpy as np 
import pandas as pd 
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import lightgbm

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


import warnings
warnings.filterwarnings("ignore")

# Read Datas & Explanation of Features & Information About Datasets

In [ ]:
dataset = pandas.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
dataset.head(10)

In [ ]:
dataset.drop("enrollee_id", axis=1, inplace=True)

## Variable Descriptions



In [ ]:
dataset.info()

In [ ]:
dataset.describe().T

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.isnull().sum().sum()

### Univariate Variable Analysis

* Categorical Variables: 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'

* Numerical Variables: 'id', 'hypertension', 'heart_disease', 'stroke'

#### Categorical Variables

In [ ]:
def bar_plot(variable):
    # get feature
    var = dataset[variable]
    # count number of categorical variable(value/sample)
    varValue = var.value_counts()
    
    # visualize
    plt.figure(figsize = (9,3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}:\n{}".format(variable,varValue))

In [ ]:
categorical = (dataset.dtypes == "object")
categorical_list = list(categorical[categorical].index)

print("Categorical variables:")
print(categorical_list)

In [ ]:
for c in categorical_list:
    bar_plot(c)

#### Numerical Variables

In [ ]:
numerical_int64 = (dataset.dtypes == "int64")
numerical_int64_list = list(numerical_int64[numerical_int64].index)

print("Categorical variables:")
print(numerical_int64_list)

In [ ]:
def plot_hist(variable):
    plt.figure(figsize = (9,3))
    plt.hist(dataset[variable], bins = 50)
    plt.xlabel(variable)
    plt.ylabel("Frequency")
    plt.title("{} distribution with hist".format(variable))
    plt.show()

In [ ]:
for n in numerical_int64_list:
    plot_hist(n)

In [ ]:
numerical_float64 = (dataset.dtypes == "float64")
numerical_float64_list = list(numerical_float64[numerical_float64].index)

print("Numerical variables:")
print(numerical_float64_list)

In [ ]:
for n in numerical_float64_list:
    plot_hist(n)

# Basic Data Analysis

In [ ]:
plt.figure(figsize=(50,50))
j = 0

for i in categorical_list:
    colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#fbdf70','#ac9fd0','#8b7470']
    
    labels = dataset[i].value_counts().index
    sizes = dataset[i].value_counts().values
    
    unique = len(dataset[i].unique())
    if(unique == 2):
        myexplode = [0.1, 0]
    if(unique == 3):
        myexplode = [0.1, 0,0]
    if(unique == 4):
        myexplode = [0.1,0,0,0]
    
    plt.subplot(4,3,j+1)
    plt.pie(sizes, labels=labels, shadow = True, startangle=90, colors=colors, autopct='%1.1f%%',textprops={'fontsize': 12})
    plt.title(f'Distribution of {i}',color = 'black',fontsize = 25)
    j += 1

In [ ]:
plt.figure(figsize=(25,15))
plt.subplot(2,3,1)
plt.title('gender = Female')
dataset.groupby('gender').target.value_counts().loc['Female'].plot(kind='bar')

plt.subplot(2,3,2)
plt.title('gender = Male')
dataset.groupby('gender').target.value_counts().loc['Male'].plot(kind='bar')

plt.subplot(2,3,3)
plt.title('gender = Other')
dataset.groupby('gender').target.value_counts().loc['Other'].plot(kind='bar')

In [ ]:
plt.figure(figsize=(25,15))
plt.subplot(2,3,1)
plt.title('relevent_experience = Has relevent experience')
dataset.groupby('relevent_experience').target.value_counts().loc['Has relevent experience'].plot(kind='bar')

plt.subplot(2,3,2)
plt.title('relevent_experience = No relevent experience')
dataset.groupby('relevent_experience').target.value_counts().loc['No relevent experience'].plot(kind='bar')

In [ ]:
plt.figure(figsize=(25,15))
plt.subplot(2,3,1)
plt.title('enrolled_university = no_enrollment')
dataset.groupby('enrolled_university').target.value_counts().loc['no_enrollment'].plot(kind='bar')

plt.subplot(2,3,2)
plt.title('enrolled_university = Full time course')
dataset.groupby('enrolled_university').target.value_counts().loc['Full time course'].plot(kind='bar')

plt.subplot(2,3,3)
plt.title('enrolled_university = Part time course')
dataset.groupby('enrolled_university').target.value_counts().loc['Part time course'].plot(kind='bar')

In [ ]:
plt.figure(figsize=(30,15))
plt.subplot(2,3,1)
plt.title('education_level = Graduate')
dataset.groupby('education_level').target.value_counts().loc['Graduate'].plot(kind='bar')

plt.subplot(2,3,2)
plt.title('education_level = Masters')
dataset.groupby('education_level').target.value_counts().loc['Masters'].plot(kind='bar')

plt.subplot(2,3,3)
plt.title('education_level = High School')
dataset.groupby('education_level').target.value_counts().loc['High School'].plot(kind='bar')

plt.subplot(2,3,4)
plt.title('education_level = Phd')
dataset.groupby('education_level').target.value_counts().loc['Phd'].plot(kind='bar')

plt.subplot(2,3,5)
plt.title('education_level = Primary School')
dataset.groupby('education_level').target.value_counts().loc['Primary School'].plot(kind='bar')

In [ ]:
plt.figure(figsize=(30,15))
plt.subplot(2,3,1)
plt.title('major_discipline = STEM')
dataset.groupby('major_discipline').target.value_counts().loc['STEM'].plot(kind='bar')

plt.subplot(2,3,2)
plt.title('major_discipline = Business Degree')
dataset.groupby('major_discipline').target.value_counts().loc['Business Degree'].plot(kind='bar')

plt.subplot(2,3,3)
plt.title('major_discipline = Arts')
dataset.groupby('major_discipline').target.value_counts().loc['Arts'].plot(kind='bar')

plt.subplot(2,3,4)
plt.title('major_discipline = Humanities')
dataset.groupby('major_discipline').target.value_counts().loc['Humanities'].plot(kind='bar')

plt.subplot(2,3,5)
plt.title('major_discipline = No Major')
dataset.groupby('major_discipline').target.value_counts().loc['No Major'].plot(kind='bar')

plt.subplot(2,3,6)
plt.title('major_discipline = Other')
dataset.groupby('major_discipline').target.value_counts().loc['Other'].plot(kind='bar')

In [ ]:
plt.figure(figsize=(30,15))
plt.subplot(2,3,1)
plt.title('company_type = Pvt Ltd')
dataset.groupby('company_type').target.value_counts().loc['Pvt Ltd'].plot(kind='bar')

plt.subplot(2,3,2)
plt.title('company_type = Funded Startup')
dataset.groupby('company_type').target.value_counts().loc['Funded Startup'].plot(kind='bar')

plt.subplot(2,3,3)
plt.title('company_type = Early Stage Startup')
dataset.groupby('company_type').target.value_counts().loc['Early Stage Startup'].plot(kind='bar')

plt.subplot(2,3,4)
plt.title('company_type = Other')
dataset.groupby('company_type').target.value_counts().loc['Other'].plot(kind='bar')

plt.subplot(2,3,5)
plt.title('company_type = Public Sector')
dataset.groupby('company_type').target.value_counts().loc['Public Sector'].plot(kind='bar')

plt.subplot(2,3,6)
plt.title('company_type = NGO')
dataset.groupby('company_type').target.value_counts().loc['NGO'].plot(kind='bar')

In [ ]:
plt.figure(figsize=(30,15))

plt.subplot(2,3,1)
sns.histplot(dataset['training_hours'], color = 'red', kde = True).set_title('training_hours Interval and Counts')

plt.subplot(2,3,2)
sns.histplot(dataset['city_development_index'], color = 'green', kde = True).set_title('city_development_index Interval and Counts')

plt.subplot(2,3,3)
sns.histplot(dataset['target'], color = 'blue', kde = True).set_title('target Interval and Counts')

# Missing Values

In [ ]:
def missing_values_percentage(feature):
    missing_values_number = dataset[feature].isnull().sum()
    if(missing_values_number > 0):
        print(f'There are {missing_values_number} missing values in "{feature}" column.')
        missing_value_percentage = 100 * missing_values_number / len(dataset['gender'])
        print(f'{missing_value_percentage:.2f}% of the "{feature}" feature are missing values')
        print("*****"*20)

In [ ]:
features = list(dataset.columns)

for c in features:
    missing_values_percentage(c)

In [ ]:
def fill_missings(feature):
    missing_values_number = dataset[feature].isnull().sum()
    if(missing_values_number > 0):
        
        missing_value_percentage = 100 * missing_values_number / len(dataset['gender'])

        if(missing_value_percentage <= 0):
            #dataset.dropna(subset = [feature], inplace = True)
            missing_values_number = dataset[feature].isnull().sum()
            missing_value_percentage = 100 * missing_values_number / len(dataset['gender'])
            print(f'There are {missing_values_number} missing values in "{feature}" column.')
            print(f'{missing_value_percentage:.2f}% of the "{feature}" feature are missing values')
        else:
            dataset[feature].fillna(method ='bfill', limit = 4, inplace = True)
            missing_values_number = dataset[feature].isnull().sum()
            if(missing_values_number > 0):
                dataset[feature].fillna(method ='ffill', limit = 4, inplace = True)
                
                missing_values_number = dataset[feature].isnull().sum()
                missing_value_percentage = 100 * missing_values_number / len(dataset['gender'])
                print(f'There are {missing_values_number} missing values in "{feature}" column.')
                print(f'{missing_value_percentage:.2f}% of the "{feature}" feature are missing values')
                
        print("*****"*20)
        

In [ ]:
for c in features:
    fill_missings(c)

In [ ]:
dataset.isnull().sum()

# Skewness

In [ ]:
dataset.agg(['skew']).T

In [ ]:
skews = ['city_development_index', 'training_hours']

In [ ]:
from scipy.stats import norm, skew, boxcox
for i in skews:
    sns.set_style('darkgrid')
    sns.distplot(dataset[i], fit = norm)
    plt.title('Skeweed')
    plt.show()
    (mu, sigma) = norm.fit(dataset[i])
    print("mu {} : {}, sigma {} : {}".format(i, mu, i, sigma))
    print()
    
    dataset[i], lam = boxcox(dataset[i])

    sns.set_style('darkgrid')
    sns.distplot(dataset[i], fit = norm)
    plt.title('Transformed')
    plt.show()
    (mu, sigma) = norm.fit(dataset[i])
    print("mu {} : {}, sigma {} : {}".format(i, mu, i, sigma))
    print()

In [ ]:
dataset.agg(['skew']).T

# One-Hot Encoding

In [ ]:
cat_var =  ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 
            'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']

for i in range (0, len(cat_var)):
    print(f'Unique Values for {cat_var[i]}', dataset[f'{cat_var[i]}'].unique())

In [ ]:
onehotencoder = OneHotEncoder()

In [ ]:
one_hot = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 
            'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']

for i in range(0, len(one_hot)):
    dataset[f'{one_hot[i]}'] = pd.Categorical(dataset[f'{one_hot[i]}'])
    dummies = pd.get_dummies(dataset[f'{one_hot[i]}'], prefix = f'{one_hot[i]}_encoded')
    dataset.drop([f'{one_hot[i]}'], axis=1, inplace=True)
    dataset = pd.concat([dataset, dummies], axis=1)

In [ ]:
dataset

# Train-Test Split

In [ ]:
X = dataset.drop(["target"],axis =1)
y = dataset["target"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101) 
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print(f'Total # of sample in whole dataset: {len(X)}')
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Total # of sample in validation dataset: {len(X_valid)}')
print(f'Total # of sample in test dataset: {len(X_test)}')

# Pipelines

In [ ]:
pipeline_GaussianNB = Pipeline([("scaler",StandardScaler()),
                     ("pipeline_GaussianNB",GaussianNB())])

pipeline_BernoulliNB = Pipeline([("scaler",StandardScaler()),
                     ("pipeline_BernoulliNB",BernoulliNB())])

pipeline_LogisticRegression = Pipeline([("scaler",StandardScaler()),
                     ("pipeline_LogisticRegression",LogisticRegression())])

pipeline_RandomForest = Pipeline([("scaler",StandardScaler()),
                     ("pipeline_RandomForest",RandomForestClassifier())])

pipeline_DecisionTree = Pipeline([("scaler",StandardScaler()),
                     ("pipeline_DecisionTree",DecisionTreeClassifier())])

pipeline_KNN = Pipeline([("scaler",StandardScaler()),
                     ("pipeline_KNN",KNeighborsClassifier())])

pipeline_GBC = Pipeline([("scaler",StandardScaler()), (
                        "pipeline_GBC",GradientBoostingClassifier())])

pipeline_SGD = Pipeline([("scaler",StandardScaler()), 
                        ("pipeline_SGD",SGDClassifier(max_iter=5000, random_state=0))])

pipeline_LGBM = Pipeline([("scaler",StandardScaler()), 
                        ("pipeline_NN",lightgbm.LGBMClassifier())])

pipelines = [pipeline_GaussianNB, pipeline_BernoulliNB, pipeline_LogisticRegression, pipeline_RandomForest, pipeline_DecisionTree, pipeline_KNN, pipeline_GBC, pipeline_SGD, pipeline_LGBM]

pipe_dict = {0: "GaussianNB", 1: "BernoulliNB", 2: "LogisticRegression",3: "RandomForestClassifier", 4: "DecisionTreeClassifier",
            5: "KNeighborsClassifier", 6: "GradientBoostingClassifier", 7:"Stochastic Gradient Descent", 8: "LGBM"}

modelNames = ["GaussianNB", 'BernoulliNB','LogisticRegression','RandomForestClassifier',
             'DecisionTreeClassifier', 'KNeighborsClassifier','GradientBoostingClassifier',
             'Stochastic Gradient Descent', 'LGBM']

i= 0
trainScores = []
validationScores = []
testScores = []

for pipe in pipelines:
    pipe.fit(X_train, y_train)
    print(f'{pipe_dict[i]}')
    print("Train Score of %s: %f     " % (pipe_dict[i], pipe.score(X_train, y_train)*100))
    trainScores.append(pipe.score(X_train, y_train)*100)
    
    print("Validation Score of %s: %f" % (pipe_dict[i], pipe.score(X_valid, y_valid)*100))
    validationScores.append(pipe.score(X_valid, y_valid)*100)
    
    print("Test Score of %s: %f      " % (pipe_dict[i], pipe.score(X_test, y_test)*100))
    testScores.append(pipe.score(X_test, y_test)*100)
    print(" ")
    
    y_predictions = pipe.predict(X_test)
    conf_matrix = confusion_matrix(y_predictions, y_test)
    print(f'Confussion Matrix: \n{conf_matrix}\n')
    
    tn = conf_matrix[0,0]
    fp = conf_matrix[0,1]
    tp = conf_matrix[1,1]
    fn = conf_matrix[1,0]

    total = tn + fp + tp + fn
    real_positive = tp + fn
    real_negative = tn + fp

    accuracy  = (tp + tn) / total # Accuracy Rate
    precision = tp / (tp + fp) # Positive Predictive Value
    recall    = tp / (tp + fn) # True Positive Rate
    f1score  = 2 * precision * recall / (precision + recall)
    specificity = tn / (tn + fp) # True Negative Rate
    error_rate = (fp + fn) / total # Missclassification Rate
    prevalence = real_positive / total
    miss_rate = fn / real_positive # False Negative Rate
    fall_out = fp / real_negative # False Positive Rate
    
    print('Evaluation Metrics:')
    print(f'Accuracy    : {accuracy}')
    print(f'Precision   : {precision}')
    print(f'Recall      : {recall}')
    print(f'F1 score    : {f1score}')
    print(f'Specificity : {specificity}')
    print(f'Error Rate  : {error_rate}')
    print(f'Prevalence  : {prevalence}')
    print(f'Miss Rate   : {miss_rate}')
    print(f'Fall Out    : {fall_out}')

    print("") 
    print(f'Classification Report: \n{classification_report(y_predictions, y_test)}\n')
    print("")

    print("*****"*20)
    i +=1

In [ ]:
plt.figure(figsize=(20,10))
sns.set_style('darkgrid')
plt.title('Train - Validation - Test Scores of Models', fontweight='bold', size = 24)

barWidth = 0.25
 
bars1 = trainScores
bars2 = validationScores
bars3 = testScores
 
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
 
plt.bar(r1, bars1, color='blue', width=barWidth, edgecolor='white', label='train', yerr=0.5,ecolor="black",capsize=10)
plt.bar(r2, bars2, color='#557f2d', width=barWidth, edgecolor='white', label='validation', yerr=0.5,ecolor="black",capsize=10, alpha = .50)
plt.bar(r3, bars3, color='red', width=barWidth, edgecolor='white', label='test', yerr=0.5,ecolor="black",capsize=10, hatch = '-')
 
modelNames = ["GaussianNB", 'BernoulliNB','LogisticRegression','RandomForestClassifier',
             'DecisionTreeClassifier', 'KNeighborsClassifier','GradientBoostingClassifier',
             'Stochastic Gradient Descent', 'LGBM']
    
plt.xlabel('Algorithms', fontweight='bold', size = 24)
plt.ylabel('Scores', fontweight='bold', size = 24)
plt.xticks([r + barWidth for r in range(len(bars1))], modelNames, rotation = 75)
 
plt.legend()
plt.show()

In [ ]:
table = pd.DataFrame({'Model': modelNames, 'Train': trainScores, 'Validation': validationScores, 'Test': testScores})
table

## Cross Validation

In [ ]:
cv_results_acc = []

for i, model in enumerate(pipelines):
    cv_score = cross_val_score(model, X_train, y_train, scoring = "accuracy", cv = 10)
    cv_results_acc.append(cv_score.mean()*100)
    print("%s: %f" % (pipe_dict[i], cv_score.mean()*100))

In [ ]:
table_cv = pd.DataFrame({'Model': modelNames, 'CV Score': cv_results_acc})
table_cv

In [ ]:
plt.figure(figsize=(20,10))
sns.set_style('darkgrid')
plt.title('CV Scores Means', fontweight='bold', size = 24)

barWidth = 0.5
 
bars2 = cv_results_acc
 
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
 
plt.bar(r2, bars2, color='#557f2d', width=barWidth, edgecolor='black',  yerr=0.5,ecolor="black",capsize=10)


modelNames = ["GaussianNB", 'BernoulliNB','LogisticRegression','RandomForestClassifier',
             'DecisionTreeClassifier', 'KNeighborsClassifier','GradientBoostingClassifier',
             'Stochastic Gradient Descent', 'Light GBM']
    
plt.xlabel('Algorithms', fontweight='bold', size = 24)
plt.ylabel('Scores', fontweight='bold', size = 24)
plt.xticks([r + barWidth for r in range(len(bars1))], modelNames, rotation = 75)
 
plt.legend()
plt.show()

# Conclusion

I made Visualization and Machine Learning Classification on this notebook. 

* If you have questions, please comment them. I will try to explain if you don't understand.
* If you liked this notebook, please let me know :)

Thank you for your time.